# Midterm Project: ETL and Data Warehousing
Madelyn Khoury (mgk5ybb) and Tiara Allard (tia4qp)

DS 2002 Spring 2023

## Design and Strategy

We chose to model bank transactions as the core business process of our data warehouse, so we designed a database schema centered around bank transactions. To see the schema we designed for this project, please look at the ReadMe of our GitHub project. 

Our schema stores information about bank transactions, bank accounts and users involved in transactions, transaction dates, and transaction locations. We were unable to find a database/dataset with all this information, so instead we combined data from multiple different data sources. This had the added benefit of allowing us to meet the requirements for importing data from a number of sources.

We combined several dummy/randomly generated datasets to build a complete data warehouse. First, we got bank transaction and account information from a .csv file stored on our local filesystem. Then, we generated user data from an API and linked it to the accounts and transactions. Finally, we imported location information from the Northwind MySQL database to represent regions in which banking transactions might have occurred.

After processing the data and computing useful fields, we formatted it into our fact and dimension tables in the final data warehouse.

## Imports and Helper Functions

In [ ]:
import sys
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install pymysql
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install uszipcode

In [ ]:
import datetime
import json
import mysql.connector
import os
import pandas as pd
import pymysql
import random
import requests
from sqlalchemy import create_engine
from uszipcode import SearchEngine, SimpleZipcode

In [2]:
def get_api_response(url, headers, params, response_type):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        # result = json.dumps(response.json(), sort_keys=True, indent=4)
        result = response.json()
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [3]:
# this helper function is inspired by part of one of the provided files, 02-Python-MySQL.ipynb
def get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    dframe = None
    try:
        conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
        sqlEngine = create_engine(conn_str, pool_recycle=3600)
        connection = sqlEngine.connect()
        try:
            dframe = pd.read_sql(sql_query, connection);
        except:
            print("Sequel query was unsuccessful.")
        connection.close()
        return dframe
    except:
        print("Unable to connect to the MySQL database.")
    return None

In [4]:
def execute_mysql_command(user_id, pwd, host_name, db_name, sql_query, use_db):
    try:
        if use_db:
            conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
        else:
            conn = pymysql.connect(host=host_name, user=user_id, password=pwd)
        cursor = conn.cursor()
        
        try:
            cursor.execute(sql_query)
            for row in cursor.fetchall():
                print(row)
            cursor.close()
        except:
            print("Cannot execute command.")
    except:
        print("Unable to connect to database.")
        
    conn.close()

In [5]:
def insert_data_to_mysql(user_id, pwd, host_name, db_name, my_dataframe, table_name, index=True):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    my_dataframe.to_sql(table_name, con=connection, schema="banks", if_exists='append', index=index)
    connection.close()

In [6]:
# this code snippet is modified from: https://www.geeksforgeeks.org/python-program-to-calculate-age-in-year/ 
def calculate_age(birth_date):
    birth_date = datetime.datetime.strptime(birth_date, '%Y-%m-%d').date()
    today = datetime.date.today()
    try:
        birthday = birth_date.replace(year = today.year)
 
    # raised when birth date is February 29 but it's not a leap year
    except ValueError:
        birthday = birth_date.replace(year = today.year,
                  month = birth_date.month + 1, day = 1) # birth date becomes march 1st
 
    if birthday > today:
        return today.year - birth_date.year - 1
    else:
        return today.year - birth_date.year

In [7]:
def get_region(state_abbreviation):
    if state_abbreviation in {"WA", "OR", "CA", "ID", "MT", "NV", "UT", "CO", "WY", "AK"}:
        return "West"
    elif state_abbreviation in {"AZ", "NM", "TX", "OK"}:
        return "Southwest"
    elif state_abbreviation in {"ND", "SD", "NE", "KS", "MN", "IA", "MO", "WI", "IL", "MI", "OH"}:
        return "Midwest"
    elif state_abbreviation in {"ME", "NH", "MA", "CT", "RI", "VT", "NY", "PA", "DE", "MD", "NJ"}:
        return "Northeast"
    else:
        return "Southeast"

In [8]:
def get_zipcode(city, state):
    search = SearchEngine()
    results = search.by_city_and_state(city, state)
    if len(results) > 0:
        return results[0].zipcode
    else:
        return None

In [9]:
def get_day(date_timestamp):
    return date_timestamp.day

In [10]:
def get_month(date_timestamp):
    return date_timestamp.month_name()

In [11]:
def get_year(date_timestamp):
    return date_timestamp.year

In [12]:
def get_week_day(date_timestamp):
    return date_timestamp.day_name()

## Loading in Data
In this section, we will be loading in the data from three sources: an API, a local filesystem, and a relational database.

### Importing Data From Local File System

The core bank transaction information that we will use came from a dataset on Kaggle (https://www.kaggle.com/datasets/apoorvwatsky/bank-transaction-data). We downloaded the data in the form of a xlsx file and will import it from the local filesystem in order to be used in our data warehouse.

In [13]:
bank_info_path = os.path.join(os.getcwd(), 'bank.xlsx')
bank_info = pd.read_excel(bank_info_path)

In [14]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


### Importing Data From API

We've chosen to use the `users` endpoint from random-data-api.com, which randomly generates data for a set of users. This will populate the Users table in our data warehouse.

In [15]:
size = 10 # only get info on 10 users for now
url = "https://random-data-api.com/api/v2/users"
querystring = {"size":size}
headers = None

# Get information from users API endpoint
users = get_api_response(url, headers, querystring, "dataframe")
users

,id,uid,password,first_name,last_name,username,email,avatar,gender,phone_number,...,address.zip_code,address.state,address.country,address.coordinates.lat,address.coordinates.lng,credit_card.cc_number,subscription.plan,subscription.status,subscription.payment_method,subscription.term
0,5491,61537405-643d-43f0-a7d9-6ec6c4e44b60,2u9jTzeU4K,Lory,Klocko,lory.klocko,lory.klocko@email.com,https://robohash.org/quietrerum.png?size=300x3...,Genderfluid,+66 1-666-306-1680,...,02787,West Virginia,United States,77.514603,97.808926,4955-5327-6007-1331,Professional,Pending,Alipay,Full subscription
1,5654,ad974045-70bc-469b-8ad6-8654da1bd4dc,jFHheKE7SY,Laurel,Parisian,laurel.parisian,laurel.parisian@email.com,https://robohash.org/numquameaquenemo.png?size...,Genderqueer,+691 925.509.8901 x3611,...,24540,Pennsylvania,United States,6.980797,-157.746293,4410-6725-5030-2840,Free Trial,Pending,Cheque,Annual
2,1455,4a65c767-f28b-470b-9b77-bce446bebaaa,z5SDcQNoal,Brandie,Carroll,brandie.carroll,brandie.carroll@email.com,https://robohash.org/eumisteut.png?size=300x30...,Bigender,+252 141-100-3277 x238,...,72119,Nebraska,United States,45.525154,174.528822,4863290938627,Premium,Idle,Bitcoins,Monthly
3,7486,dc09e57c-7b91-46bb-b3d0-48945ee8d402,dliXutbPTS,Vance,Oberbrunner,vance.oberbrunner,vance.oberbrunner@email.com,https://robohash.org/corruptivoluptatumet.png?...,Polygender,+994 (532) 068-9133,...,16235-0089,Oklahoma,United States,49.356868,147.451810,5558-4559-4474-2700,Premium,Blocked,Debit card,Monthly
4,7036,63741a3a-99a5-4bb7-8ff6-852d76cfd291,qBjCxYrJKe,Thea,Yost,thea.yost,thea.yost@email.com,https://robohash.org/voluptatemaperiamsed.png?...,Bigender,+30 (727) 974-7141 x57322,...,99123,Virginia,United States,16.789436,57.453642,5112-9011-7639-3851,Premium,Pending,WeChat Pay,Monthly
5,5507,cd1dc6df-37a6-4311-adf5-7c4c8e0a2c13,HpTQamUw6I,Roy,Pacocha,roy.pacocha,roy.pacocha@email.com,https://robohash.org/nihilrerumdolores.png?siz...,Male,+387 1-243-651-0468,...,39061,Nebraska,United States,7.862350,110.228244,4386-5356-8449-3928,Silver,Active,Alipay,Monthly
6,9367,0af43b1d-4872-4cce-9e6b-f699606aba96,hc8qoJpQfT,Bryon,Kassulke,bryon.kassulke,bryon.kassulke@email.com,https://robohash.org/nullavoluptatemquia.png?s...,Genderfluid,+298 356-448-2228,...,23007,Nebraska,United States,-66.114910,19.447769,4764955091381,Professional,Pending,Alipay,Annual
7,8324,fa03d53e-5fee-47fc-807b-61e04b7b7667,Z3mJnFg6s8,Leonel,Collier,leonel.collier,leonel.collier@email.com,https://robohash.org/eumdoloresvoluptas.png?si...,Polygender,+1-649 (103) 242-2823 x4132,...,75237-4698,Tennessee,United States,57.156625,-48.918066,4817-7212-7763-6557,Free Trial,Idle,Cash,Payment in advance
8,786,fb59430f-fa92-4f27-94f0-b65274114dc3,5fajYw9Ghi,Angelic,Okuneva,angelic.okuneva,angelic.okuneva@email.com,https://robohash.org/nostrumsitdolores.png?siz...,Female,+44 960.978.3311,...,86203,Iowa,United States,-12.002387,115.023442,4130-3985-7722-4549,Platinum,Blocked,Cash,Full subscription
9,7219,69f29d0d-4005-4fa2-9570-bcbddf550885,aQhB37g5bf,Maryrose,Smith,maryrose.smith,maryrose.smith@email.com,https://robohash.org/nesciuntanimiest.png?size...,Bigender,+687 1-822-161-9514 x8307,...,00193,Arkansas,United States,20.809360,57.993477,4531573377827,Business,Active,Bitcoins,Monthly


### Importing Data From Relational Database

To get location data that could represent locations in which bank transactions were completed, we've decided to import information about the shipping location of orders from the `orders` table in the Northwind database. In our data warehouse, this information will represent the location in which a customer instigated a banking transaction; perhaps it could represent the location of physical branches of the bank.

In [16]:
# define variables to set up connection to mySQL database
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "northwind"

First we must get the location-related data from the `orders` table.

In [17]:
sql_query = """
    SELECT ship_address, ship_city, ship_state_province, ship_zip_postal_code, ship_country_region from orders;
"""

In [18]:
locations_info = get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query)

## Transforming/Cleaning Up the Data
In this section, we will be transforming, cleaning, and doing transformations on the data, as well as separating it into several tables that we can easily import into our data warehouse.

### Transforming the Location Data

We got location info from the Northwind database, but we must remove duplicate values so that we have a table of unique locations.

In [19]:
locations_info = locations_info.drop_duplicates()

It appears that the Northwind database didn't store actual zip codes, but instead put 99999 in for every row. So, we will fill in the table with the correct zip code for each city listed. Additionally, we will add another column to the table which will identify the region of the United States that the location is in.

In [20]:
locations_info["zipcode"] = locations_info.apply(lambda row: get_zipcode(row["ship_city"], row["ship_state_province"]), axis=1)
locations_info.drop(["ship_zip_postal_code"], axis = 1, inplace = True)

In [21]:
locations_info["region"] = locations_info.apply(lambda row: get_region(row["ship_state_province"]), axis=1)
locations_info

,ship_address,ship_city,ship_state_province,ship_country_region,zipcode,region
0,789 27th Street,Las Vegas,NV,USA,89101,West
1,123 4th Street,New York,NY,USA,10001,Northeast
2,123 12th Street,Las Vegas,NV,USA,89101,West
3,123 8th Street,Portland,OR,USA,97201,West
5,789 29th Street,Denver,CO,USA,80202,West
6,123 3rd Street,Los Angelas,CA,USA,90001,West
7,123 6th Street,Milwaukee,WI,USA,53202,Midwest
8,789 28th Street,Memphis,TN,USA,38103,Southeast
10,123 10th Street,Chicago,IL,USA,60601,Midwest
11,123 7th Street,Boise,ID,USA,83702,West


### Transforming the User Data

To transform the user data, all we had to do was drop some columns of the dataframe. As we can see by looking at the columns of users, there is a lot of superfluous information.

In [22]:
users.columns

Index(['id', 'uid', 'password', 'first_name', 'last_name', 'username', 'email',
       'avatar', 'gender', 'phone_number', 'social_insurance_number',
       'date_of_birth', 'employment.title', 'employment.key_skill',
       'address.city', 'address.street_name', 'address.street_address',
       'address.zip_code', 'address.state', 'address.country',
       'address.coordinates.lat', 'address.coordinates.lng',
       'credit_card.cc_number', 'subscription.plan', 'subscription.status',
       'subscription.payment_method', 'subscription.term'],
      dtype='object')

In [23]:
users.drop(['employment.title', 'employment.key_skill', 'uid','avatar', 'social_insurance_number', 'subscription.plan', 'subscription.payment_method', 'subscription.status', 'subscription.term', 'address.city', 'address.street_name', 'address.street_address', 'address.zip_code', 'address.state', 'address.country', 'address.coordinates.lat', 'address.coordinates.lng'], axis = 1, inplace = True)

We will also calculate the age of each user, that way we have calculations stored in our OLAP database and don't have to compute them on the fly.

In [24]:
users["age"] = users.apply(lambda row: calculate_age(row["date_of_birth"]), axis=1)

In [25]:
users

,id,password,first_name,last_name,username,email,gender,phone_number,date_of_birth,credit_card.cc_number,age
0,5491,2u9jTzeU4K,Lory,Klocko,lory.klocko,lory.klocko@email.com,Genderfluid,+66 1-666-306-1680,1989-12-09,4955-5327-6007-1331,33
1,5654,jFHheKE7SY,Laurel,Parisian,laurel.parisian,laurel.parisian@email.com,Genderqueer,+691 925.509.8901 x3611,1984-04-14,4410-6725-5030-2840,38
2,1455,z5SDcQNoal,Brandie,Carroll,brandie.carroll,brandie.carroll@email.com,Bigender,+252 141-100-3277 x238,1977-11-16,4863290938627,45
3,7486,dliXutbPTS,Vance,Oberbrunner,vance.oberbrunner,vance.oberbrunner@email.com,Polygender,+994 (532) 068-9133,1995-08-14,5558-4559-4474-2700,27
4,7036,qBjCxYrJKe,Thea,Yost,thea.yost,thea.yost@email.com,Bigender,+30 (727) 974-7141 x57322,1998-11-17,5112-9011-7639-3851,24
5,5507,HpTQamUw6I,Roy,Pacocha,roy.pacocha,roy.pacocha@email.com,Male,+387 1-243-651-0468,1961-01-03,4386-5356-8449-3928,62
6,9367,hc8qoJpQfT,Bryon,Kassulke,bryon.kassulke,bryon.kassulke@email.com,Genderfluid,+298 356-448-2228,1982-01-15,4764955091381,41
7,8324,Z3mJnFg6s8,Leonel,Collier,leonel.collier,leonel.collier@email.com,Polygender,+1-649 (103) 242-2823 x4132,1991-11-30,4817-7212-7763-6557,31
8,786,5fajYw9Ghi,Angelic,Okuneva,angelic.okuneva,angelic.okuneva@email.com,Female,+44 960.978.3311,1968-02-01,4130-3985-7722-4549,55
9,7219,aQhB37g5bf,Maryrose,Smith,maryrose.smith,maryrose.smith@email.com,Bigender,+687 1-822-161-9514 x8307,2003-05-03,4531573377827,19


### Transforming the Account Information

Reviewing the bank_info table we made, we can see that it has columns relating not just to a single transaction, but also to the date and account associated with the transaction.

In [26]:
bank_info.columns

Index(['Account No', 'DATE', 'TRANSACTION DETAILS', 'CHQ.NO.', 'VALUE DATE',
       'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', '.'],
      dtype='object')

We will separate this data into three tables: a Transactions fact table, an Accounts dimension, and a Date dimension. 
The bank transaction fact table can store the new balance of an account after a transaction is completed, but we want the bank account table to store current info about each account-- information which is independent of any one transaction. So, we will take the most recent balance for each account and create a "Current Balance" field in the Accounts table.  Since the rows of the spreadsheet were sorted in order of transaction date, then the most recent balance for each account will be the balance in the last-occurring transaction.

In [103]:
account_info = bank_info[["Account No", "BALANCE AMT"]]
account_info

,Account No,BALANCE AMT
0,409000611074',1.000000e+06
1,409000611074',2.000000e+06
2,409000611074',2.500000e+06
3,409000611074',5.500000e+06
4,409000611074',6.000000e+06
...,...,...
116196,409000362497',-1.901902e+09
116197,409000362497',-1.901602e+09
116198,409000362497',-1.901302e+09
116199,409000362497',-1.901412e+09


In [104]:
account_info = account_info.drop_duplicates(subset=["Account No"], keep="last") # keep only the last record for each account
account_info = account_info.reset_index(drop=True) # I'm also going to reset the indices to start from 0

Our schema design includes another field in the accounts table: the ID of the customer who holds this account. We will randomly select users from our users table to act as the "holders" of these accounts. We will also randomly select the type of each account from a list of options.

In [105]:
account_info["User ID"] = account_info.apply(lambda row: random.randint(1, users.shape[0]), axis=1)

In [106]:
bank_account_types = ["Checking", "Savings", "Money market (MMA)", "Certificate of deposit (CD)"]
account_info["Account Type"] = account_info.apply(lambda row: bank_account_types[random.randint(0, len(bank_account_types)-1)], axis=1)

In [107]:
account_info = account_info.rename(columns={"Account No": "account_no", "BALANCE AMT": "balance", "User ID": "user_id", "Account Type": "account_type"})
account_info

,account_no,balance,user_id,account_type
0,409000611074',4.622000e+05,7,Savings
1,409000493201',7.435833e+05,3,Checking
2,409000425051',-3.567348e+08,1,Savings
3,409000405747',-5.482675e+08,8,Checking
4,409000438611',-5.479193e+08,6,Savings
5,409000493210',-5.463146e+08,2,Savings
6,409000438620',-5.399631e+08,10,Money market (MMA)
7,1196711',-1.586916e+09,8,Money market (MMA)
8,1196428',-1.687234e+09,8,Checking
9,409000362497',-1.901417e+09,9,Savings


### Transforming the Date Information

The other aspect of transactions that we want to put in its own dimension table is the date of each transaction. We will generate a date entry for each date that occurs in the table of transaction information, then link the two tables together.

In [108]:
# get a list of unique dates from the table of transaction info
date_info = bank_info.copy().drop_duplicates(subset=["DATE"])
date_info = date_info.reset_index(drop=True) # reset the indices to start from 0
date_info.drop([ 'Account No','TRANSACTION DETAILS', 'CHQ.NO.','VALUE DATE', 'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', '.'], axis = 1, inplace = True)
date_info

,DATE
0,2017-06-29
1,2017-07-05
2,2017-07-18
3,2017-08-01
4,2017-08-16
...,...
1289,2017-12-25
1290,2018-04-02
1291,2018-04-29
1292,2018-05-12


In [109]:
date_info["day"] = date_info.apply(lambda row: get_day(row["DATE"]), axis=1)

In [110]:
date_info["month"] = date_info.apply(lambda row: get_month(row["DATE"]), axis=1)

In [111]:
date_info["year"] = date_info.apply(lambda row: get_year(row["DATE"]), axis=1)

In [112]:
date_info["week_day"] = date_info.apply(lambda row: get_week_day(row["DATE"]), axis=1)

In [113]:
date_info = date_info.rename(columns={"DATE": "date"})
date_info

,date,day,month,year,week_day
0,2017-06-29,29,June,2017,Thursday
1,2017-07-05,5,July,2017,Wednesday
2,2017-07-18,18,July,2017,Tuesday
3,2017-08-01,1,August,2017,Tuesday
4,2017-08-16,16,August,2017,Wednesday
...,...,...,...,...,...
1289,2017-12-25,25,December,2017,Monday
1290,2018-04-02,2,April,2018,Monday
1291,2018-04-29,29,April,2018,Sunday
1292,2018-05-12,12,May,2018,Saturday


### Transforming the Transaction Data

Finally, we can clean up the transaction data!

In [114]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


In [142]:
transaction_info = bank_info.copy()
transaction_info.drop(['CHQ.NO.','VALUE DATE', '.'], axis = 1, inplace = True)
transaction_info = transaction_info.rename(columns={"Account No": "account_no", "DATE": "date", "TRANSACTION DETAILS": "details", "WITHDRAWAL AMT": "withdrawal_amount", "DEPOSIT AMT": "deposit_amount", "BALANCE AMT": "balance"})
transaction_info

,account_no,date,details,withdrawal_amount,deposit_amount,balance
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,1000000.0,1.000000e+06
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,1000000.0,2.000000e+06
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,500000.0,2.500000e+06
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,3000000.0,5.500000e+06
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,500000.0,6.000000e+06
...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,117934.30,NaN,-1.901902e+09
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,300000.0,-1.901602e+09
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,300000.0,-1.901302e+09
116199,409000362497',2019-03-05,IMPS 05-03-20194C,109868.65,NaN,-1.901412e+09


In [116]:
locations_info

,ship_address,ship_city,ship_state_province,ship_country_region,zipcode,region
0,789 27th Street,Las Vegas,NV,USA,89101,West
1,123 4th Street,New York,NY,USA,10001,Northeast
2,123 12th Street,Las Vegas,NV,USA,89101,West
3,123 8th Street,Portland,OR,USA,97201,West
5,789 29th Street,Denver,CO,USA,80202,West
6,123 3rd Street,Los Angelas,CA,USA,90001,West
7,123 6th Street,Milwaukee,WI,USA,53202,Midwest
8,789 28th Street,Memphis,TN,USA,38103,Southeast
10,123 10th Street,Chicago,IL,USA,60601,Midwest
11,123 7th Street,Boise,ID,USA,83702,West


## Creating and Populating the Data Warehouse
In this section, we will create a new SQL database with all the tables specified in our schema. We'll then populate it with the data tables that we created earlier.


### Creating the Database
First, we create a new database -- we'll call it "banks" -- to serve as our data warehouse.

In [153]:
# create banks database
queries = ['SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0;', 
'SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0;',
'SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE=\'TRADITIONAL,ALLOW_INVALID_DATES\';',
'DROP SCHEMA IF EXISTS `banks` ;',
'CREATE SCHEMA IF NOT EXISTS `banks` DEFAULT CHARACTER SET latin1 ;',
'USE `banks` ;']

for query in queries:
    execute_mysql_command(user_id, pwd, host_name, "banks", query, False)

Next, we'll make all the tables in the database.

In [154]:
transaction_info.columns

Index(['account_no', 'date', 'details', 'withdrawal_amount', 'deposit_amount',
       'balance'],
      dtype='object')

In [155]:
db_name = "banks"
queries = ["""
CREATE TABLE IF NOT EXISTS `banks`.`users` (
  `id` INT(15) NOT NULL AUTO_INCREMENT,
  `password` VARCHAR(50) NULL DEFAULT NULL,
  `first_name` VARCHAR(50) NULL DEFAULT NULL,
  `last_name` VARCHAR(50) NULL DEFAULT NULL,
  `username` VARCHAR(50) NULL DEFAULT NULL,
  `email` VARCHAR(50) NULL DEFAULT NULL,
  `gender` VARCHAR(50) NULL DEFAULT NULL,
  `phone_number` VARCHAR(50) NULL DEFAULT NULL,
  `date_of_birth` VARCHAR(25) NULL DEFAULT NULL,
  `credit_card.cc_number` VARCHAR(25) NULL DEFAULT NULL,
  `age` INT(3) NULL DEFAULT NULL,
  PRIMARY KEY (`id`))
  ENGINE = InnoDB
  DEFAULT CHARACTER SET = utf8;
  """,
  """
  CREATE TABLE IF NOT EXISTS `banks`.`locations` (
  `loc_id` INT(15) NOT NULL AUTO_INCREMENT,
  `ship_address` VARCHAR(50) NOT NULL,
  `ship_city` VARCHAR(50) NULL DEFAULT NULL,
  `ship_state_province` VARCHAR(50) NULL DEFAULT NULL,
  `ship_country_region` VARCHAR(15) NULL DEFAULT NULL,
  `zipcode` VARCHAR(15) NULL DEFAULT NULL,
  `region` VARCHAR(50) NULL DEFAULT NULL,
  PRIMARY KEY (`loc_id`))
  ENGINE = InnoDB
  DEFAULT CHARACTER SET = utf8;
  """,
  """
  CREATE TABLE IF NOT EXISTS `banks`.`dates` (
  `date` DATETIME ,
  `day` INT(2) NULL DEFAULT NULL,
  `month` VARCHAR(15) NULL DEFAULT NULL,
  `year` VARCHAR(5) NULL DEFAULT NULL,
  `week_day` VARCHAR(15) NULL DEFAULT NULL,
  PRIMARY KEY (`date`))
  ENGINE = InnoDB
  DEFAULT CHARACTER SET = utf8;
  """,
  """
  CREATE TABLE IF NOT EXISTS `banks`.`accounts` (
  `account_no` VARCHAR(15) ,
  `balance` VARCHAR(15) NULL DEFAULT NULL,
  `user_id` INT(5) NULL DEFAULT NULL,
  `account_type` VARCHAR(30) NULL DEFAULT NULL,
  PRIMARY KEY (`account_no`),
  CONSTRAINT `user_id`
  FOREIGN KEY (`user_id`)
  REFERENCES `banks`.`users` (`id`)
  ON DELETE NO ACTION
  ON UPDATE NO ACTION)
  ENGINE = InnoDB
  DEFAULT CHARACTER SET = utf8;
  """,
  """
  CREATE TABLE IF NOT EXISTS `banks`.`transactions` (
  `transaction_id` INT(15) NOT NULL AUTO_INCREMENT,
  `account_no` VARCHAR(15) NULL DEFAULT NULL,
  `balance` VARCHAR(15) NULL DEFAULT NULL,
  `withdrawal_amount` VARCHAR(15) NULL DEFAULT NULL,
  `deposit_amount` VARCHAR(15) NULL DEFAULT NULL,
  `details` VARCHAR(50) NULL DEFAULT NULL,
  `date` DATETIME ,
  PRIMARY KEY (`transaction_id`),
  CONSTRAINT `accounts`
    FOREIGN KEY (`account_no`)
    REFERENCES `banks`.`accounts` (`account_no`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB
DEFAULT CHARACTER SET = utf8;
"""]

for query in queries:
    execute_mysql_command(user_id, pwd, host_name, db_name, query, True)

### Populating the Database

In [156]:
insert_data_to_mysql(user_id, pwd, host_name, db_name, date_info, "dates", index=False)

In [157]:
insert_data_to_mysql(user_id, pwd, host_name, db_name, users, "users", index=False)

In [158]:
account_info

,account_no,balance,user_id,account_type
0,409000611074',4.622000e+05,7,Savings
1,409000493201',7.435833e+05,3,Checking
2,409000425051',-3.567348e+08,1,Savings
3,409000405747',-5.482675e+08,8,Checking
4,409000438611',-5.479193e+08,6,Savings
5,409000493210',-5.463146e+08,2,Savings
6,409000438620',-5.399631e+08,10,Money market (MMA)
7,1196711',-1.586916e+09,8,Money market (MMA)
8,1196428',-1.687234e+09,8,Checking
9,409000362497',-1.901417e+09,9,Savings


In [159]:
insert_data_to_mysql(user_id, pwd, host_name, db_name, locations_info, "locations", index=False)

In [160]:
test_insert_query = "SELECT * from users"
test_insert = get_mysql_dataframe(user_id, pwd, host_name, db_name, test_insert_query)
test_insert

,id,password,first_name,last_name,username,email,gender,phone_number,date_of_birth,credit_card.cc_number,age
0,1,2u9jTzeU4K,Lory,Klocko,lory.klocko,lory.klocko@email.com,Genderfluid,+66 1-666-306-1680,1989-12-09,4955-5327-6007-1331,33
1,2,jFHheKE7SY,Laurel,Parisian,laurel.parisian,laurel.parisian@email.com,Genderqueer,+691 925.509.8901 x3611,1984-04-14,4410-6725-5030-2840,38
2,3,z5SDcQNoal,Brandie,Carroll,brandie.carroll,brandie.carroll@email.com,Bigender,+252 141-100-3277 x238,1977-11-16,4863290938627,45
3,4,dliXutbPTS,Vance,Oberbrunner,vance.oberbrunner,vance.oberbrunner@email.com,Polygender,+994 (532) 068-9133,1995-08-14,5558-4559-4474-2700,27
4,5,qBjCxYrJKe,Thea,Yost,thea.yost,thea.yost@email.com,Bigender,+30 (727) 974-7141 x57322,1998-11-17,5112-9011-7639-3851,24
5,6,HpTQamUw6I,Roy,Pacocha,roy.pacocha,roy.pacocha@email.com,Male,+387 1-243-651-0468,1961-01-03,4386-5356-8449-3928,62
6,7,hc8qoJpQfT,Bryon,Kassulke,bryon.kassulke,bryon.kassulke@email.com,Genderfluid,+298 356-448-2228,1982-01-15,4764955091381,41
7,8,Z3mJnFg6s8,Leonel,Collier,leonel.collier,leonel.collier@email.com,Polygender,+1-649 (103) 242-2823 x4132,1991-11-30,4817-7212-7763-6557,31
8,9,5fajYw9Ghi,Angelic,Okuneva,angelic.okuneva,angelic.okuneva@email.com,Female,+44 960.978.3311,1968-02-01,4130-3985-7722-4549,55
9,10,aQhB37g5bf,Maryrose,Smith,maryrose.smith,maryrose.smith@email.com,Bigender,+687 1-822-161-9514 x8307,2003-05-03,4531573377827,19


In [161]:
insert_data_to_mysql(user_id, pwd, host_name, db_name, account_info, "accounts", index=False)

In [162]:
insert_data_to_mysql(user_id, pwd, host_name, db_name, transaction_info, "transactions", index=False)

In [163]:
test_insert_query = "SELECT * from transactions limit 10"
test_insert = get_mysql_dataframe(user_id, pwd, host_name, db_name, test_insert_query)
test_insert

,transaction_id,account_no,balance,withdrawal_amount,deposit_amount,details,date
0,1,409000611074',1000000,None,1000000,TRF FROM Indiaforensic SERVICES,2017-06-29
1,2,409000611074',2000000,None,1000000,TRF FROM Indiaforensic SERVICES,2017-07-05
2,3,409000611074',2500000,None,500000,FDRL/INTERNAL FUND TRANSFE,2017-07-18
3,4,409000611074',5500000,None,3000000,TRF FRM Indiaforensic SERVICES,2017-08-01
4,5,409000611074',6000000,None,500000,FDRL/INTERNAL FUND TRANSFE,2017-08-16
5,6,409000611074',6500000,None,500000,FDRL/INTERNAL FUND TRANSFE,2017-08-16
6,7,409000611074',7000000,None,500000,FDRL/INTERNAL FUND TRANSFE,2017-08-16
7,8,409000611074',7500000,None,500000,FDRL/INTERNAL FUND TRANSFE,2017-08-16
8,9,409000611074',8000000,None,500000,FDRL/INTERNAL FUND TRANSFE,2017-08-16
9,10,409000611074',8500000,None,500000,FDRL/INTERNAL FUND TRANSFE,2017-08-16


## Executing SQL Queries
Finally, in this section, we will execute several SQL queries that aggregate data from at least three of the data tables in our data warehouse.

First, we will join information from the dates dimension table, accounts dimension table, and transactions fact table to get information about all the transactions that occurred on a Thursday.

In [164]:
sql_query = "SELECT details, week_day, first_name FROM transactions JOIN dates ON transactions.date = dates.date JOIN accounts ON transactions.account_no = accounts.account_no JOIN users ON accounts.user_id = users.id WHERE week_day = 'Thursday';"
thursday_transactions = get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query)
thursday_transactions

,details,week_day,first_name
0,NEFT/BARBH16098696964/MOT,Thursday,Lory
1,NEFT/STBP916112924171/UNI,Thursday,Lory
2,NEFT/BKIDN16126554823/NAS,Thursday,Lory
3,NEFT/IBKL160512086896/SUN,Thursday,Lory
4,NEFT/SBIN716133295639/PRA,Thursday,Lory
...,...,...,...
20441,AEPS PAY CR ADJIndiaforensic_R 2,Thursday,Maryrose
20442,AEPS PAY CR ADJIndiaforensic 270,Thursday,Maryrose
20443,MICRO ATM WDL FEE DT28021,Thursday,Maryrose
20444,MICRO ATM INC DATED 28021,Thursday,Maryrose


We will also get the sum of all the withdrawals that have been taken out of every bank account.

In [167]:
sql_query = "SELECT SUM(withdrawal_amount), transactions.account_no, first_name, last_name FROM transactions JOIN accounts ON transactions.account_no = accounts.account_no JOIN users ON accounts.user_id = users.id GROUP BY account_no; "
withdrawals = get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query)
withdrawals

,SUM(withdrawal_amount),account_no,first_name,last_name
0,3.845102e+08,409000425051',Lory,Klocko
1,1.006049e+08,409000493210',Laurel,Parisian
2,9.537793e+07,409000493201',Brandie,Carroll
3,4.705551e+09,409000438611',Roy,Pacocha
4,1.453974e+08,409000611074',Bryon,Kassulke
5,6.848283e+10,1196428',Leonel,Collier
6,4.692584e+10,1196711',Leonel,Collier
7,4.203179e+08,409000405747',Leonel,Collier
8,1.019351e+11,409000362497',Angelic,Okuneva
9,1.719608e+10,409000438620',Maryrose,Smith
